# Nooksack Groundwater-Surface Water Model Coupling

### Install packages and libraries

In [ ]:
# Packages not on HydroShare R kernal 
install.packages('hydroGOF')
install.packages('writexl')
install.packages('doParallel')

In [ ]:
# silence the library loading messages
library(data.table)
library(writexl)

suppressMessages(library(dplyr))
suppressMessages(library(reshape2))
suppressMessages(library(hydroGOF))
suppressMessages(library(doParallel))

### File management and decompression

#### Only need to run the cell below the first time the data is downloaded from HydroShare to this HPC workspace. See working directory above.

unzip('modelruns_1952WRIA1_081418.zip')

### Open a new terminal, change directory, and unzip all model files
cd 
'/home/jovyan/work/notebooks/data/fb5e45f7bfea4765a445a190b809cbdb/fb5e45f7bfea4765a445a190b809cbdb/data/contents'

Run
> gunzip modelruns_1952WRIA1_081418/*/*

In [ ]:
# assign Project 2018 as the current directory
Project2018 <- getwd()
WMoff2018_Models <- file.path(Project2018, 'modelruns_1952WRIA1_081418/modelruns_1952WRIA1_110912_WMoff')
WMon2012_Models <- file.path(Project2018, 'modelruns_1952WRIA1_081418/modelruns_1952WRIA1_110912_WMon')

In [ ]:
# set working directory
setwd(WMoff2018_Models)

### Read in files and data

In [ ]:
list.files()

# 1.0 Recharge Calculations

In [ ]:
# read topsbd (mod: changed to fread for spead and added data.table=T)
topsbd <- fread('topsbd_v8.txt', skip = 1, header = T, data.table=T)
colnames(topsbd)
head(topsbd,10)

In [ ]:
# generate TimeStep by subbasin matrices (mod: added 'mean' as the aggregation function)
Zbar <- topsbd %>% select(TimeStep, Basin, Afrac, WTDepth_mm) %>% mutate(afrac_test = Afrac*WTDepth_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)
PET <- topsbd %>% select(TimeStep, Basin, Afrac, Pet_mm) %>% mutate(afrac_test = Afrac*Pet_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)
AET <- topsbd %>% select(TimeStep, Basin, Afrac, Aet_mm) %>% mutate(afrac_test = Afrac*Aet_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)
Rain <- topsbd %>% select(TimeStep, Basin, Afrac, Prec_mm) %>% mutate(afrac_test = Afrac*Prec_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)
SatEx <- topsbd %>% select(TimeStep, Basin, Afrac, SatEx_mm) %>% mutate(afrac_test = Afrac*SatEx_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)
InfEx <- topsbd %>% select(TimeStep, Basin, Afrac, InfEx_mm) %>% mutate(afrac_test = Afrac*InfEx_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)
SurfRO <- topsbd %>% select(TimeStep, Basin, Afrac, SurfRo_mm) %>% mutate(afrac_test = Afrac*SurfRo_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)
SoilStore <- topsbd %>% select(TimeStep, Basin, Afrac, SoilStore_mm) %>% mutate(afrac_test = Afrac*SoilStore_mm) %>% dcast(TimeStep~Basin, value.var='afrac_test', fun.aggregate=sum)

In [ ]:
# load Recharge inputs
#Basins <- read.table("basin.txt", header=T, row.names=1) # mod: added row.names=1
Precipitation_mm <- Rain
Depth_to_Water_mm <- Zbar
Evaporation_mm = fread("Evaporation_mm.txt", header=T)
Soil_storage_mm <- SoilStore

# Make if statement here
Surface_runoff_cms = fread("TotalRunoff_noWithdrawal_cms.txt", header=T)

In [ ]:
#dim(Basins)
dim(Precipitation_mm)
dim(Evaporation_mm)
dim(Depth_to_Water_mm) # matrices sourced from Basin are selected subset of basins
dim(Surface_runoff_cms)

## Print individual text files of water budget variables
See variable names for text file naming

In [ ]:
# print these outputs from topsbd_v8.txt
for (mat in c('InfEx', 'PET', 'AET', 'SatEx', 'Rain', 'Zbar', 'SoilStore', 'Surface_runoff_cms')){
    write.table(mat %>% get() %>% data.frame(), file = paste0(mat, '.txt'), col.names = T, row.names = F, append = F)
    print(paste(mat, 'printed to', paste0(mat, '.txt')))
}

### Functions for aggregation

In [ ]:
# function to calculate the water year
wtr_yr <- function(dates, start_month=10) {
    # Convert dates into POSIXlt
    dates.posix = as.POSIXlt(as.character(dates), format = '%Y%m%d')

    # Calculate offset based on the month of the year
    offset = ifelse(dates.posix$mon >= start_month, 1, 0)

    # adjust the current year to the appropriate Water year
    adj.year = dates.posix$year + 1900 + offset # year + adjustment for date origin + offset
    return(adj.year)
}


# function to read in the time series files 
findmeta <- function(file){
    # index last row of metadata, depends on use of "Ver#" start to header line
    meta_end <- grep('^Ver', readLines(file))

    # metadata
    meta <- readLines(file)[1:meta_end] %>% 
      gsub("^\\t+|\\t+$|^\\s+|\\s+$|^ ","",.) %>% # removing leading and trailing spaces and tabs
      gsub("\\t+"," ",.) # separate tab-separated metadata

    print(paste("metadata headers for", file))
    return(meta)
}


# function to read in .dat format output files
read_dat <- function(file, meta){
    # column names based on tab separation
    col.Names <- meta %>% gsub("\\t+$","",.) %>%
        strsplit(' |\t') %>% 
        tail(1) %>%
        unlist() %>%
        .[-c(1,2)] %>%
        .[.!=""]

    # arrange the data
    dat <- read.csv(file, sep='', skip = length(meta), header = F)
    nonNA <- colnames(dat)[which(colSums(is.na(dat)) != nrow(dat))]
    dat <- dat[colnames(dat) %in% nonNA] %>% data.table()
    setnames(dat, colnames(dat), col.Names)

    return(dat)
}

### Algorithm to Calculate Recharge

In [ ]:
# functionalize the timesteploop
timesteploop <- function(TimeStep, basin_subset, totalRecharge_depot, Precipitation_mm, Evaporation_mm, 
                         Surface_runoff_cms, Soil_storage_mm, totalRecharge=0){
        
    # for every TimeStep
    for(ts in TimeStep){
        
        # for each catchment
        for (CatchID in basin_subset[['Basins']]){
            Drainage = as.character(basin_subset[Basins==CatchID,'Drainages'])
            
            ### Look up P, E, and SR values for this daily timestep and convert to inches
            P <- Precipitation_mm[ts, CatchID]*0.0393701	#Precipitation (in)
            E <- as.numeric(Evaporation_mm[ts, Drainage, with=F])*0.0393701	#Evaporation (in)
            SR <- as.numeric(Surface_runoff_cms[ts, Drainage, with=F])*61024*60*60*24/Basinpars[CatchID, "BasinArea_in2"] #Surface runoff/Basin area (in)

            if(ts==1){
                CDW <- as.numeric(0) ### Calculate change in depth to water (in)
                CSS <- as.numeric(0) ### Calculate change in soil storage term (in)
            } else {
                # take the difference between the current water depth and the previous water depth at this Drainage
                CDW <- (Depth_to_Water_mm[ts, CatchID] - Depth_to_Water_mm[ts-1, CatchID])*0.0393701
                CSS <- (Soil_storage_mm[ts, CatchID] - Soil_storage_mm[ts-1, CatchID])*0.0393701
            }

            ### Calculate daily recharge
            dailyRecharge = P-E-SR+CSS

            ### Calculate running total of daily recharge
            totalRecharge = totalRecharge + dailyRecharge

            # store the data
            totalRecharge_depot <- rbindlist(list(totalRecharge_depot,
                                                  data.table(CatchID,ts,dailyRecharge,totalRecharge)),
                                             use.names = T, fill = T)
        }
    }
    fwrite(totalRecharge_depot, file='totalRecharge_depot.txt', sep='\t', col.names=T, row.names=F, append=F)
    return(totalRecharge_depot)
}



In [ ]:
# functionalize the timesteploop
timestepper <- function(TimeStep, CatchID, 
                        basin_subset, totalRecharge_basin, Precipitation_mm, Evaporation_mm,
                        Surface_runoff_cms, Soil_storage_mm, totalRecharge=0){
    # for every TimeStep
    for(ts in TimeStep){
        Drainage = as.character(basin_subset[Basins==CatchID,'Drainages'])

        #Look up P, E, and SR values for this daily timestep and convert to inches
        
        # Precipitation (in)
        P <- Precipitation_mm[ts, CatchID]*0.0393701

        # Evaporation (in)
        E <- as.numeric(Evaporation_mm[ts, Drainage, with=F])*0.0393701
        
        # Surface runoff/Basin area (in)
        SR <- as.numeric(Surface_runoff_cms[ts, Drainage, with=F])*61024*60*60*24/Basinpars[CatchID, "BasinArea_in2"]

        if(ts==1){
            CDW <- as.numeric(0) # Calculate change in depth to water (in)
            CSS <- as.numeric(0) # Calculate change in soil storage term (in)
        } else {
            # take the difference between the current water depth and the previous water depth at this Drainage
            CDW <- (Depth_to_Water_mm[ts, CatchID] - Depth_to_Water_mm[ts-1, CatchID])*0.0393701
            CSS <- (Soil_storage_mm[ts, CatchID] - Soil_storage_mm[ts-1, CatchID])*0.0393701
        }

        # Calculate daily recharge
        dailyRecharge = P-E-SR+CSS

        # Calculate running total of daily recharge
        totalRecharge = totalRecharge + dailyRecharge

        # store the data
        totalRecharge_basin <- rbindlist(list(totalRecharge_basin,
                                              data.table(CatchID,ts,dailyRecharge,totalRecharge)),
                                         use.names = T, fill = T)
    }
    return(totalRecharge_basin)
}


# 2.0 Water Budget Calculations

### Load input files and list basins selected for analysis

In [ ]:
Project2018
WMoff2018_Models

In [ ]:
### Load input files (if they have not already been read in above) ###
if(!"Basinpars" %in% ls()){
    Basinpars = read.table("basinpars.txt", sep=',', header=T)
}
if(!"TimeSteps" %in% ls()){
    TimeSteps <- read.table('DateTime_yyyymmdd_hhmmss.txt', header = T)
}
if(!"Precipitation_mm" %in% ls()){
    Precipitation_mm = read.table("Precipitation_mm.txt", header=T)
}
if(!"Evaporation_mm" %in% ls()){
    Evaporation_mm = read.table("Evaporation_mm.txt", header=T)
}
if(!"Surface_runoff_cms" %in% ls()){
    Surface_runoff_cms = read.table("Surface_runoff_cms.txt", header=T)
}
if(!"Soil_storage_mm" %in% ls()){
    Soil_storage_mm = read.table("SoilStore.txt", header=T)
}
if(!"Depth_to_Water_mm" %in% ls()){
    Depth_to_Water_mm = read.table("Zbar.txt", header=T)
}

# compute basin area
Basinpars['BasinArea_in2'] <- Basinpars[,'direct_area']*(0.0393701^2)
# compute basin_subset
basin_subset <- data.table(Basins=colnames(Precipitation_mm)[!colnames(Precipitation_mm) %in% c('TimeStep')])
basin_subset[,Drainages:=paste0('Drainage',Basins)]


### Set up to read model inputs

In [ ]:
ModelSet_folder <- getwd()
print(paste('start', ModelSet_folder))

# set start date and end date of rain year
Rain_start <- as.numeric(19000101) #as.numeric(20061001)
Rain_end <- as.numeric(20150930)
calib_start <- as.numeric(20030613)
calib_end <- as.numeric(20051231)
valid_start <- as.numeric(20060101)
valid_end <- as.numeric(20160930)

In [ ]:
# set target files
#WM on  #used with WMon
readus_WMon <-c('FlowAtStreamNodes_cms.txt',
                'UserDemand_cms.txt',
                'UserWithdrawal_cms.txt',
                'TotalRunoff_cms.txt', 
                'UserDeficit_cms.txt',
                'Precipitation_mm.txt',
                'Evaporation_mm.txt',
                'DateTime_yyyymmdd_hhmmss.txt',
                'Artificial_Drainage.txt',
                'basinpars.txt',
                'StreamFlowLinks.txt',
                'DrainageID.txt',
                'DrainageInfo.txt',
                'MonthlyDemandFraction.txt',
                'user.txt')

#WMoff
readus_WMoff<-c('FlowAtStreamNodes_cms.txt',
                'TotalRunoff_noWithdrawal_cms.txt',
                'Precipitation_mm.txt',
                'Evaporation_mm.txt',
                'DateTime_yyyymmdd_hhmmss.txt',
                'Artificial_Drainage.txt',
                'basinpars.txt',
                'StreamFlowLinks.txt',
                'DrainageID.txt',
                'DrainageInfo.txt')



### Select list of files based on the working directory (WM on/off)

In [ ]:
if('WMon' %in% getwd()){
    readus<-readus_WMon
} else {
    readus<-readus_WMoff
}

In [ ]:
# readus files - data tables named after the names of the files
for (i in readus){
    assign(i, i %>% fread(header=T, data.table=T))
    print(paste('reading', i))
}

### Set up time and unit dimensions

In [ ]:
# Abstract the Year, Month, and range for the Water Year period
DateTime_water_year.txt <- DateTime_yyyymmdd_hhmmss.txt %>%
    mutate(year = substring(yyyymmdd, 1,4) %>% as.numeric(),
           month = substring(yyyymmdd, 5, 6) %>% as.numeric(),
           day = substring(yyyymmdd, 7, 8) %>% as.numeric(),
           water_year = wtr_yr(yyyymmdd, start_month = 9)) %>%
    filter(yyyymmdd >= Rain_start,
           yyyymmdd <= Rain_end) %>% 
    arrange(TimeStep) %>%
    data.table()

# calib
DateTime_calib.txt <- DateTime_yyyymmdd_hhmmss.txt %>%
    mutate(year = substring(yyyymmdd, 1,4) %>% as.numeric(),
           month = substring(yyyymmdd, 5, 6) %>% as.numeric(),
           day = substring(yyyymmdd, 7, 8) %>% as.numeric(),
           water_year = wtr_yr(yyyymmdd, start_month = 9)) %>%
    filter(yyyymmdd >= calib_start,
           yyyymmdd <= calib_end) %>% 
    arrange(TimeStep) %>%
    data.table()


# validation
DateTime_valid.txt <- DateTime_yyyymmdd_hhmmss.txt %>%
    mutate(year = substring(yyyymmdd, 1,4) %>% as.numeric(),
           month = substring(yyyymmdd, 5, 6) %>% as.numeric(),
           day = substring(yyyymmdd, 7, 8) %>% as.numeric(),
           water_year = wtr_yr(yyyymmdd, start_month = 9)) %>%
    filter(yyyymmdd >= valid_start,
           yyyymmdd <= valid_end) %>% 
    arrange(TimeStep) %>%
    data.table()


# convert mm to inches
Precipitation_in.txt <- Precipitation_mm.txt %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(./25.4)) %>%
    merge(., DateTime_water_year.txt, by = 'TimeStep') %>% 
    arrange(TimeStep) %>%
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Drainage')) %>%
    data.table()


Evaporation_in.txt <- Evaporation_mm.txt %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(./25.4)) %>%
    merge(., DateTime_water_year.txt, by = 'TimeStep') %>% 
    arrange(TimeStep) %>%
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Drainage')) %>%
    data.table()


# convert cms to cfs: 1cms = 35.3147 cfs
FlowAtStreamNodes_cfs.txt <- FlowAtStreamNodes_cms.txt %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    merge(., DateTime_water_year.txt, by = 'TimeStep') %>% 
    mutate(FlowAtOutlet = Node1) %>% # 
    arrange(TimeStep) %>% 
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Node')) %>%
    data.table()

Artificial_Drainage_cfs.txt <- Artificial_Drainage.txt %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    merge(., DateTime_water_year.txt, by = 'TimeStep') %>% 
    arrange(TimeStep) %>% 
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Drainage')) %>%
    data.table()

if('WMon' %in% getwd()){
    # convert cms to gal/day: 1cms * 264.172 gal/m^3 * 86400 s/day = 23688461 gal/day
    UserDemand_gpd.txt <- UserDemand_cms.txt %>%
        mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*264.172*86400)) %>%
        merge(., DateTime_water_year.txt, by = 'TimeStep') %>% 
        arrange(TimeStep) %>% 
        select(TimeStep, yyyymmdd, year, month, day, water_year, contains('User')) %>%
        data.table()

    UserWithdrawal_gpd.txt <- UserWithdrawal_cms.txt %>%
        mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*264.172*86400)) %>%
        merge(., DateTime_water_year.txt, by = 'TimeStep') %>% 
        arrange(TimeStep) %>% 
        select(TimeStep, yyyymmdd, year, month, day, water_year, contains('User')) %>%
        data.table()
}


# find meta lines for rain.dat
rain.dat_meta <- findmeta('rain.dat')

# read in rain.dat
rain.dat <- read_dat('rain.dat', rain.dat_meta)

# apply conversion from mm to inches. Code-base taken from Precipitation.
# convert mm to inches
rain_in_obs.dat <- rain.dat %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(./25.4)) %>%
    merge(., DateTime_water_year.txt, by.x = 'Date', by.y = 'yyyymmdd') %>% 
    arrange(TimeStep) %>%
    select(-c(Date, year, day, Hour, hhmmss))

### Run the Recharge Calculation for Subbasins

In [ ]:
# takes roughly 3 minutes for 21652 time units for 16 basins

# start time
ptm <- Sys.time()

# initialize a cluster with n workers
c1 <- makeCluster(20)
registerDoParallel(c1)

# for each catchment
totalRecharge_depot <- foreach(CatchID = basin_subset[['Basins']], 
                               .packages = c('dplyr','data.table'),
                               .combine=function(x,y)rbindlist(list(x,y))) %dopar% {
                                   
                                   totalRecharge_output <- timestepper(TimeSteps$TimeStep, 
                                                                       CatchID, 
                                                                       basin_subset, 
                                                                       totalRecharge_basin=data.table(), 
                                                                       Precipitation_mm, 
                                                                       Evaporation_mm, 
                                                                       Surface_runoff_cms, 
                                                                       Soil_storage_mm, 
                                                                       totalRecharge=0)
                               }
                               
# write out the depot
fwrite(totalRecharge_depot, file='totalRecharge_depot.txt', sep='\t', col.names=T, row.names=F, append=F)


# stop cluster
stopCluster(c1)
print(Sys.time()-ptm)

In [ ]:
dim(totalRecharge_depot)

In [ ]:
head(totalRecharge_depot)
#totalRecharge_depot[(CatchID==3),]

### Calculate Aggregated Recharge

In [ ]:
# additional calculations
totalRecharge_summary <- totalRecharge_depot %>%
    merge(., DateTime_water_year.txt, by.x='ts', by.y = 'TimeStep') %>%
  
    # MonthlyRecharge total
    group_by(month, water_year, CatchID) %>%
    mutate(monthlyRecharge = sum(dailyRecharge)) %>%

    # AnnualRecharge total
    group_by(water_year, CatchID) %>%
    mutate(annualRecharge = sum(dailyRecharge)) %>%

    # WaterYear average AnnualRecharge
    group_by(water_year, CatchID) %>%
    mutate(averageMonthlyRechargeByWateryear = mean(monthlyRecharge)) %>%
    data.table() %>%

    group_by(CatchID) %>%
    mutate(averageAnnualRecharge = mean(annualRecharge)) %>%
    data.table()

# print summary
fwrite(totalRecharge_summary, paste0(getwd(),'totalRecharge_summary.txt'), sep = '\t', row.names = F, col.names = T, append = F)
print(Sys.time()-ptm)

unzip(paste0(getwd(),'totalRecharge_depot.txt.gz'))

In [ ]:
# (length(TimeSteps$TimeStep)/365) * as.numeric(tdiff) / 60 # to hours
# length(basin_subset$Basin)
# length(TimeSteps$TimeStep)

In [ ]:
# ### Calculate average annual recharge (inches) for the basin ###
# ptm <- Sys.time()

# totalRecharge_depot <- timesteploop(TimeSteps$TimeStep[1:365], 
#                                     basin_subset,
#                                     totalRecharge_depot=data.table(), 
#                                     Precipitation_mm, 
#                                     Evaporation_mm, 
#                                     Surface_runoff_cms, 
#                                     Soil_storage_mm, 
#                                     totalRecharge=0)

# (tdiff = ptm - Sys.time())

In [ ]:
# totalRecharge_depot[(CatchID==6),]

In [ ]:
# totalRecharge_depot <- fread(paste0(getwd(),'/totalRecharge_depot.txt'), sep = '\t', header = T)

# # additional calculations
# totalRecharge_depot <- totalRecharge_depot %>%
#     merge(., DateTime_water_year.txt, by.x='ts', by.y = 'TimeStep') %>%
  
#     # MonthlyRecharge total
#     group_by(month, water_year, CatchID) %>%
#     mutate(monthlyRecharge = sum(dailyRecharge)) %>%

#     # AnnualRecharge total
#     group_by(water_year, CatchID) %>%
#     mutate(annualRecharge = sum(dailyRecharge)) %>%

#     # WaterYear average AnnualRecharge
#     group_by(water_year, CatchID) %>%
#     mutate(averageMonthlyRechargeByWateryear = mean(monthlyRecharge)) %>%
#     data.table() %>%

#     group_by(CatchID) %>%
#     mutate(averageAnnualRecharge = mean(annualRecharge)) %>%
#     data.table()

# # print
# fwrite(totalRecharge_depot, paste0(getwd(),'totalRecharge_summary.txt'), sep = '\t', row.names = F, col.names = T, append = F)
# print(paste("Finished printing the RechargeCalculator within", getwd()))


### Calculate Monthly Summaries

In [ ]:
# monthly average for the 9 water years
monthlySummary_Precipitation_in.txt <- Precipitation_in.txt %>%
    # group by TimeStep and Month
    select(-c(TimeStep, yyyymmdd, year, day)) %>%
    melt(., id.vars = c('month', 'water_year'), variable.name = 'Drainages', value.name = 'Precipitation') %>%

    # calculate total by month, year, and drainage
    group_by(month, water_year, Drainages) %>%
    summarise(sumByMonthyByWateryearByDrainage = sum(Precipitation)) %>%

    # calculate mean by month, year, and drainage
    group_by(month, Drainages) %>%
    summarise(meanByMonthByDrainage = mean(sumByMonthyByWateryearByDrainage)) %>%

    # calculate mean and sd by month
    group_by(month) %>%
    mutate(meanByMonth = mean(meanByMonthByDrainage),
           sdByMonth = sd(meanByMonthByDrainage)) %>%

    # reformat the table to wide for each DrainageID
    dcast(., month + meanByMonth + sdByMonth ~ Drainages, value.var = 'meanByMonthByDrainage') %>%
    data.table()


# monthly average for the 9 water years
monthlySummary_Evaporation_in.txt <- Evaporation_in.txt %>%
    # group by TimeStep and Month
    select(-c(TimeStep, yyyymmdd, year, day)) %>%
    melt(., id.vars = c('month', 'water_year'), variable.name = 'Drainages', value.name = 'Evaporation') %>%

    # calculate total by month, year, and drainage
    group_by(month, water_year, Drainages) %>%
    summarise(sumByMonthyByWateryearByDrainage = sum(Evaporation)) %>%

    # calculate mean by month, year, and drainage
    group_by(month, Drainages) %>%
    summarise(meanByMonthByDrainage = mean(sumByMonthyByWateryearByDrainage)) %>%

    # calculate mean and sd by month
    group_by(month) %>%
    mutate(meanByMonth = mean(meanByMonthByDrainage),
           sdByMonth = sd(meanByMonthByDrainage)) %>%

    # reformat the table to wide for each DrainageID
    dcast(., month + meanByMonth + sdByMonth ~ Drainages, value.var = 'meanByMonthByDrainage') %>%
    data.table()

# the average and sd of the average Drainages
monthlySummary_Artificial_Drainage_cfs.txt <- Artificial_Drainage_cfs.txt %>%
    # group by TimeStep and Month
    select(-c(TimeStep, yyyymmdd, year, day)) %>%
    melt(., id.vars = c('month','water_year'), variable.name = 'Drainages', value.name = 'Artificial_Drainages') %>%

    # calculate the sum of artificial drainage for each month and wateryear
    group_by(month, water_year) %>%
    mutate(sumByMonthByWateryear = sum(Artificial_Drainages)) %>%

    # calculate mean by month and drainage
    group_by(month, Drainages) %>%
    mutate(meanByMonthByDrainage = mean(Artificial_Drainages)) %>%

    # calculate mean and sd by month
    group_by(month) %>%
    mutate(meanByMonth = mean(meanByMonthByDrainage),
           sdByMonth = sd(meanByMonthByDrainage),
           meanOfSumByMonth = mean(sumByMonthByWateryear)) %>%

    # reformat the table to wide for each DrainageID
    select(month, meanOfSumByMonth, meanByMonth, sdByMonth, Drainages, meanByMonthByDrainage) %>%
    unique() %>% # meanByMonthByDrainage should be the most stringent - get rid of duplicate rows
    dcast(., month + meanOfSumByMonth + meanByMonth + sdByMonth ~ Drainages, value.var = 'meanByMonthByDrainage') %>%
    data.table()


# the average and sd of the average Node
monthlySummary_FlowAtStreamNodes_cfs.txt <- FlowAtStreamNodes_cfs.txt %>%
    # group by TimeStep and Month
    select(-c(TimeStep, yyyymmdd, year, day, water_year)) %>%
    melt(., id.vars = c('month'), variable.name = 'Node', value.name = 'FlowAtStreamNodes') %>%

    # calculate mean by month and node
    group_by(month, Node) %>%
    summarise(meanByMonthByNode = mean(FlowAtStreamNodes)) %>%

    # calculate mean and sd by month
    group_by(month) %>%
    mutate(meanByMonth = mean(meanByMonthByNode),
           sdByMonth = sd(meanByMonthByNode)) %>%

    # reformat the table to wide for each NodeID
    dcast(., month+meanByMonth+sdByMonth~Node, value.var = 'meanByMonthByNode') %>%
    select(month, meanByMonth, sdByMonth, Node1, Node4, Node28) %>%
    data.table()

In [ ]:
if('WMon' %in% getwd()){

    # the average and sd of the average User
    monthlySummary_UserDemand_gpd.txt <- UserDemand_gpd.txt %>%
        # group by TimeStep and Month
        select(-c(TimeStep, yyyymmdd, year, day, water_year)) %>%
        melt(., id.vars = c('month'), variable.name = 'User', value.name = 'UserDemand_gpd') %>%
        mutate(UserType = UserType_table[match(User, user),UserType]) %>%

        # calculate mean by month, and UserType
        group_by(month, UserType) %>%
        mutate(meanByMonthByUserType = mean(UserDemand_gpd)) %>%

        # calculate mean and sd by month
        group_by(month) %>%
        mutate(meanByMonth = mean(UserDemand_gpd),
               sdByMonth = sd(UserDemand_gpd)) %>%

        # reformat the table to wide for each UserType
        select(-c(User, UserDemand_gpd)) %>%
        dcast(., month + meanByMonth + sdByMonth ~ UserType, fun = mean, value.var = c('meanByMonthByUserType')) %>%
        data.table()


    # the average monthly sum for each UserType
    monthlySums_UserDemand_gpd.txt <- UserDemand_gpd.txt %>%
        # group by month and water_year
        select(-c(TimeStep, yyyymmdd, year, day)) %>%
        melt(., id.vars = c('month','water_year'), variable.name = 'User', value.name = 'UserDemand_gpd') %>%
        mutate(UserType = UserType_table[match(User, user),UserType]) %>%

        # calculate sum by month, water_year, and UserType
        group_by(month, water_year, UserType) %>%
        mutate(UserDemand_gpm = sum(UserDemand_gpd)) %>%

        # calculate mean of the sums by month, and UserType
        group_by(month, UserType) %>%
        mutate(mean_UserDemand_gpm = mean(UserDemand_gpm)) %>%

        # calculate 
        group_by(month) %>%
        mutate(sumByMonth = sum(UserDemand_gpd),
               meanByMonth = mean(UserDemand_gpd),
               sdByMonth = sd(UserDemand_gpd),
               varByMonth = var(UserDemand_gpd)) %>%

        # reformat the table to wide for each UserType
        select(-c(User, water_year, UserDemand_gpd)) %>%
        dcast(., month + sumByMonth + meanByMonth + sdByMonth + varByMonth ~ UserType, fun = mean, value.var = c('mean_UserDemand_gpm')) %>%
        data.table()


    # the average and sd of the average User
    monthlySummary_UserWithdrawal_gpd.txt <- UserWithdrawal_gpd.txt %>%
        # group by TimeStep and Month
        select(-c(TimeStep, yyyymmdd, year, day, water_year)) %>%
        melt(., id.vars = c('month'), variable.name = 'User', value.name = 'UserWithdrawal_gpd') %>%
        mutate(UserType = UserType_table[match(User, user),UserType]) %>%

        # calculate mean by month, and UserType
        group_by(month, UserType) %>%
        mutate(meanByMonthByUserType = mean(UserWithdrawal_gpd)) %>%

        # calculate mean and sd by month
        group_by(month) %>%
        mutate(meanByMonth = mean(UserWithdrawal_gpd),
               sdByMonth = sd(UserWithdrawal_gpd)) %>%

        # reformat the table to wide for each UserType
        select(-c(User, UserWithdrawal_gpd)) %>%
        dcast(., month + meanByMonth + sdByMonth ~ UserType, fun = mean, value.var = c('meanByMonthByUserType')) %>%
        data.table()


    # the average monthly sum for each UserType
    monthlySums_UserWithdrawal_gpd.txt <- UserWithdrawal_gpd.txt %>%
        # group by TimeStep and Month
        select(-c(TimeStep, yyyymmdd, year, day)) %>%
        melt(., id.vars = c('month','water_year'), variable.name = 'User', value.name = 'UserWithdrawal_gpd') %>%
        mutate(UserType = UserType_table[match(User, user),UserType]) %>%

        # calculate mean by month, water_year, and UserType
        group_by(month, water_year, UserType) %>%
        mutate(UserWithdrawal_gpm = sum(UserWithdrawal_gpd)) %>%

        # calculate mean of the sums by month, and UserType
        group_by(month, UserType) %>%
        mutate(mean_UserWithdrawal_gpm = mean(UserWithdrawal_gpm)) %>%

        # calculate 
        group_by(month) %>%
        mutate(sumByMonth = sum(UserWithdrawal_gpd),
               meanByMonth = mean(UserWithdrawal_gpd),
               sdByMonth = sd(UserWithdrawal_gpd),
               varByMonth = var(UserWithdrawal_gpd)) %>%

        # reformat the table to wide for each UserType
        select(-c(User, water_year, UserWithdrawal_gpd)) %>%
        dcast(., month + sumByMonth + meanByMonth + sdByMonth + varByMonth ~ UserType, fun = mean, value.var = c('mean_UserWithdrawal_gpm')) %>%
        data.table()


    # monthly average for the 9 water years
    # monthlySummary_rain_in.dat <- rain_in_obs.dat %>%
    #     # group by TimeStep and Month
    #     melt(., id.vars = c('TimeStep', 'water_year', 'month'), variable.name = 'zone_code', value.name = 'rain_in_obs') %>%
    #     mutate(zone_code = paste0('zone_',zone_code)) %>%

    #     # calculate total by month, water_year, and zone_code
    #     group_by(month, water_year, zone_code) %>%
    #     mutate(sumByMonthByWateryearByZonecode = sum(rain_in_obs)) %>%

    #     # calculate mean by month, and drainage
    #     group_by(month, zone_code) %>%
    #     mutate(mean_rain_in_obs = mean(sumByMonthByWateryearByZonecode)) %>%

    #     # calculate mean and sd by month
    #     group_by(month) %>%
    #     mutate(meanSumByMonth = mean(sumByMonthByWateryearByZonecode),
    #            meanByMonth = mean(rain_in_obs),
    #            sdByMonth = sd(rain_in_obs)) %>%

    #     # reformat the table to wide for each DrainageID
    #     select(-c(TimeStep, water_year, rain_in_obs, sumByMonthByWateryearByZonecode)) %>%
    #     dcast(., month+meanSumByMonth+meanByMonth+sdByMonth~zone_code, fun=mean, value.var = c('mean_rain_in_obs')) %>%
    #     data.table()
}

### Calculate Annual Summaries

In [ ]:
# annual average for the 9 water years
yearlySummary_Precipitation_in.txt <- Precipitation_in.txt %>%
    # group by TimeStep and Month
    select(-c(TimeStep, yyyymmdd, year, day)) %>%
    melt(., id.vars = c('month', 'water_year'), variable.name = 'Drainages', value.name = 'Precipitation') %>%

    # calculate total by month, wateryear, and drainage
    group_by(month, water_year, Drainages) %>%
    summarise(sumByMonthByWateryearByDrainage = sum(Precipitation)) %>%

    # calculate mean by wateryear, and drainage
    group_by(water_year, Drainages) %>%
    summarise(meanByWateryearByDrainage = sum(sumByMonthByWateryearByDrainage)) %>%

    # calculate mean and sd by month
    group_by(water_year) %>%
    mutate(meanByWateryear = mean(meanByWateryearByDrainage),
           sdByWateryear = sd(meanByWateryearByDrainage)) %>%

    # reformat the table to wide for each DrainageID
    dcast(., water_year + meanByWateryear + sdByWateryear ~ Drainages, value.var = 'meanByWateryearByDrainage') %>%
    data.table()


# annual average for the 9 water years
yearlySummary_Evaporation_in.txt <- Evaporation_in.txt %>%
    # group by TimeStep and Month
    select(-c(TimeStep, yyyymmdd, year, day)) %>%
    melt(., id.vars = c('month', 'water_year'), variable.name = 'Drainages', value.name = 'Evaporation') %>%

    # calculate total by month, wateryear, and drainage
    group_by(month, water_year, Drainages) %>%
    summarise(sumByMonthByWateryearByDrainage = sum(Evaporation)) %>%

    # calculate mean by wateryear, and drainage
    group_by(water_year, Drainages) %>%
    summarise(meanByWateryearByDrainage = sum(sumByMonthByWateryearByDrainage)) %>%

    # calculate mean and sd by month
    group_by(water_year) %>%
    mutate(meanByWateryear = mean(meanByWateryearByDrainage),
           sdByWateryear = sd(meanByWateryearByDrainage)) %>%

    # reformat the table to wide for each DrainageID
    dcast(., water_year + meanByWateryear + sdByWateryear ~ Drainages, value.var = 'meanByWateryearByDrainage') %>%
    data.table()


if('WMon' %in% getwd()){
    # the average annual sum for each UserType
    yearlySums_UserDemand_gpd.txt <- UserDemand_gpd.txt %>%
        # group by month and water_year
        select(-c(TimeStep, yyyymmdd, year, month, day)) %>%
        melt(., id.vars = c('water_year'), variable.name = 'User', value.name = 'UserDemand_gpd') %>%
        mutate(UserType = UserType_table[match(User, user),UserType]) %>%

        # calculate sum by water_year, and UserType
        group_by(water_year, UserType) %>%
        mutate(UserDemand_gpy = sum(UserDemand_gpd)) %>%

        # calculate sum by water_year
        group_by(water_year) %>%
        mutate(sumByWateryear = sum(UserDemand_gpd)) %>%

        # reformat the table to wide for each UserType
        select(-c(UserDemand_gpd, User)) %>%
        dcast(., water_year + sumByWateryear ~ UserType, fun = mean, value.var = c('UserDemand_gpy')) %>%
        data.table()



    # the average annual sum for each UserType
    yearlySums_UserWithdrawal_gpd.txt <- UserWithdrawal_gpd.txt %>%
        # group by month and water_year
        select(-c(TimeStep, yyyymmdd, year, month, day)) %>%
        melt(., id.vars = c('water_year'), variable.name = 'User', value.name = 'UserWithdrawal_gpd') %>%
        mutate(UserType = UserType_table[match(User, user),UserType]) %>%

        # calculate sum by water_year, and UserType
        group_by(water_year, UserType) %>%
        mutate(UserWithdrawal_gpy = sum(UserWithdrawal_gpd)) %>%

        # calculate sum by water_year
        group_by(water_year) %>%
        mutate(sumByWateryear = sum(UserWithdrawal_gpd)) %>%

        # reformat the table to wide for each UserType
        select(-c(UserWithdrawal_gpd, User)) %>%
        dcast(., water_year + sumByWateryear ~ UserType, fun = mean, value.var = c('UserWithdrawal_gpy')) %>%
        data.table()
}




# # annual average for the 9 water years
# yearlySummary_rain_in_obs.dat <- rain_in_obs.dat %>%
#     # group by TimeStep and Month
#     select(-c(TimeStep)) %>%
#     melt(., id.vars = c('month', 'water_year'), variable.name = 'zone_code', value.name = 'rain_in_obs') %>%
#     mutate(zone_code = paste0('zone_',zone_code)) %>%

#     # calculate total by month, wateryear, and drainage
#     group_by(month, water_year, zone_code) %>%
#     mutate(sumByMonthByWateryearByZonecode = sum(rain_in_obs)) %>%

#     # calculate mean by wateryear, and drainage
#     group_by(water_year, zone_code) %>%
#     mutate(meanByWateryearByZonecode = mean(sumByMonthByWateryearByZonecode)) %>%

#     # calculate mean and sd by month
#     group_by(water_year) %>%
#     mutate(sumByWateryear = sum(rain_in_obs),
#            meanByWateryear = mean(sumByMonthByWateryearByZonecode),
#            sdByWateryear = sd(sumByMonthByWateryearByZonecode)) %>%

#     # reformat the table to wide for each DrainageID
#     select(-c(month, rain_in_obs, sumByMonthByWateryearByZonecode)) %>%
#     dcast(., water_year + sumByWateryear + meanByWateryear + sdByWateryear ~ zone_code, fun = mean, value.var = c('meanByWateryearByZonecode')) %>%
#     data.table()

### Calculate Artificial Drainage Flows and User Demand

In [ ]:
# Calculate the Ratios
AD_FASN_Summary <- Artificial_Drainage_cfs.txt %>%
    # group by TimeStep and Month
    select(-c(yyyymmdd, year, month, day, water_year)) %>%
    melt(., id.vars = c('TimeStep'), variable.name = 'Drainages', value.name = 'Artificial_Drainages') %>%

    # calculate the sum of artificial drainage for each month and wateryear
    group_by(TimeStep) %>%
    summarise(sumArtDrainByTimeStep = sum(Artificial_Drainages)) %>%

    # merge with FlowAtStreamNodes
    merge(., FlowAtStreamNodes_cfs.txt %>% 
            select(TimeStep, water_year, month, Node1) %>% 
            group_by(water_year, month) %>% 
            mutate(monthlyMeanNode1 = mean(Node1)),
          all=F, fill=T) %>%
    mutate(ratio_ArtDrain_Node1 = sumArtDrainByTimeStep/Node1,
           ratio_ArtDrain_monthlyMeanNode1 = sumArtDrainByTimeStep/monthlyMeanNode1) %>%
    data.table()

# UserTypes (Currently numbered like this as of 9/14/2016)
UserType_table <- rbindlist(list(
    data.table(user = paste0('User', 1:19), UserType = 'SelfSupply_US'),
    data.table(user = paste0('User', 20:31), UserType = 'SelfSupply_CAN'),
    data.table(user = paste0('User', 32:80), UserType = 'PWS_US'),
    data.table(user = paste0('User', 81:89), UserType = 'Commercial_US'),
    data.table(user = paste0('User', 90:100), UserType = 'Commercial_CAN'),
    data.table(user = paste0('User', 101:115), UserType = 'Dairy_US'),
    data.table(user = paste0('User', c(116:126,130:140,142,145:161)), UserType = 'Irrigation_US'),
    data.table(user = paste0('User', c(127, 128, 129, 141, 143, 144)), UserType = 'Irrigation_CAN'),
    data.table(user = paste0('User', 162:163), UserType = 'Reservoir_US')), use.names = T, fill = T)

### Calculate Flow Model Ouputs and Streamflow Observations

In [ ]:

# find meta lines for streamflow_calibration.dat
streamflow_calibration.dat_meta <- findmeta('streamflow_calibration.dat')
col.Names <- streamflow_calibration.dat_meta %>% gsub("\\t+$","",.) %>% strsplit(' |\t') %>% tail(1) %>% unlist() %>% .[-c(1,2)] %>% .[.!=""]


# read in streamflow_calibration.dat using readlines approach. read_dat was not working due to extra space characters.
# streamflow_calibration.dat <- read_dat('streamflow_calibration.dat', streamflow_calibration.dat_meta)
dat <- readLines('streamflow_calibration.dat', skip = length(streamflow_calibration.dat_meta)) %>%
    .[-c(1:4)] %>% 
    gsub('^[[:space:]]+|[[:space:]]+$','', .) %>% 
    gsub('[[:space:]]+','\t', .) %>% 
    strsplit(., '\t') %>% 
    do.call(rbind, .) %>% 
    data.table()

nonNA <- colnames(dat)[which(colSums(is.na(dat)) != nrow(dat))]
dat <- dat %>% select(nonNA)
setnames(dat, colnames(dat), paste0('SF',col.Names))
streamflow_calibration.dat <- dat %>% 
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(as.numeric(.))) %>% data.table()


# convert cms to cfs: 1cms = 35.3147 cfs
FlowAtStreamNodes_calib_cfs.txt <- FlowAtStreamNodes_cms.txt %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    merge(., DateTime_calib.txt, by = 'TimeStep') %>% 
    mutate(FlowAtOutlet = Node1) %>% # 
    arrange(TimeStep) %>% 
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Node')) %>%
    data.table()


# convert cms to cfs: 1cms = 35.3147 cfs
FlowAtStreamNodes_valid_cfs.txt <- FlowAtStreamNodes_cms.txt %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    merge(., DateTime_valid.txt, by = 'TimeStep', all.y = T) %>% 
    mutate(FlowAtOutlet = Node1) %>% # 
    arrange(TimeStep) %>% 
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Node')) %>%
    data.table()


# multiple flowatstreamnodes
# FlowAtStreamNodes_calib_cfs.txt
# FlowAtStreamNodes_valid_cfs.txt
# FlowAtStreamNodes_cfs.txt

# compare node4 (streamflow_calibration1) and node28 (streamflow_calibration2)
# the comparison between FlowAtStreamNodes (model output) and streamflow_calibration (obs)

# rename the metadata file
streamflow_calibration_calib_cfs.dat_meta <- streamflow_calibration.dat_meta

# adjust streamflow_calibration to cfs and filter for the calibration date range
streamflow_calibration_calib_cfs.dat <- streamflow_calibration.dat %>% 
    # convert cms to cfs
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    #mutate(SF152 = ifelse(SF152 == -999*35.3147, NA, SF152), 
    #       SF236 = ifelse(SF236 == -999*35.3147, NA, SF236)) %>%

    # add date sequence and convert to the %Y%m%d format
    mutate(date = seq(from = as.Date('20030613', format = '%Y%m%d'), by = 'day', length.out = nrow(streamflow_calibration.dat)), 
           date = format(date, '%Y%m%d')) %>%

    # filter for the respective model run dates
    filter(date %in% FlowAtStreamNodes_calib_cfs.txt[,yyyymmdd])


# # PATCH NEEDED: location observation files to use for imputation
# # obs data update for column 1
# BertrandStreamflow <- read.table(paste(ObsStreamflowData,'BertrandStreamflow.txt', sep='/'), sep = '\t', header = T) %>% 
#     mutate(Date = format(as.Date(Date, format = '%M/%d/%Y'), '%Y%m%d')) %>%
#     setnames(., colnames(.), c('Date', 'Flow_cfs')) %>%
#     data.table()
# CanadaStreamflow <- read.table(paste(ObsStreamflowData,'Daily__Sep-15-2016_10_14_39PM__ddf.txt', sep='/'), sep = '\t', header = T) %>% 
#     mutate(Date = format(as.Date(Date, format = '%M/%d/%Y'), '%Y%m%d')) %>%
#     data.table()


# # PATCH NEEDED: impute missing values using reference nodes
# # add the missing values
# streamflow_calibration_valid_cfs.dat <- streamflow_calibration_valid_cfs.dat %>%
#     mutate(SF152 = ifelse(is.na(SF152), BertrandStreamflow[match(date, Date), Flow_cfs], SF152),
#            SF236 = ifelse(is.na(SF236), CanadaStreamflow[match(date, Date), Value]*35.3147, SF236)) %>%
#     mutate(date = as.integer(date)) %>%
#     data.table()
# FlowAtStreamNodes_valid_cfs.txt <- FlowAtStreamNodes_valid_cfs.txt[yyyymmdd %in% streamflow_calibration_valid_cfs.dat[,date],]


# # PATCH NEEDED: compute nash-suttcliffe coefficient using reference nodes
# # operations linked to a reference node have been removed (e.g., SF152 and SF 236)
# NSEc1 <- NSE(sim = FlowAtStreamNodes_calib_cfs.txt[,Node4], obs = streamflow_calibration_calib_cfs.dat[,SF152], na.rm=T)
# NSEc2 <- NSE(sim = FlowAtStreamNodes_calib_cfs.txt[,Node28], obs = streamflow_calibration_calib_cfs.dat[,SF236], na.rm=T)


### Compile Table of Contents and Print Spreadsheet

In [ ]:
# compile TableofContents
TableofContents.txt <- rbindlist(list(
    data.table(fileName='TableofContents.txt', desc='table of contents'),
    data.table(fileName='DateTime_water_year.txt', desc='Time steps and datetime reference with water year'),
    data.table(fileName='UserType_table', desc='list of simulated users'),
    
    data.table(fileName='Precipitation_in.txt', desc='Precipitation in inches'),
    data.table(fileName='Evaporation_in.txt', desc='Evapotranspiration in inches'),
    data.table(fileName='FlowAtStreamNodes_cfs.txt', desc='Streamflow at Node in cubic feet per second'),
    data.table(fileName='Artificial_Drainage_cfs.txt', desc='Artificial drainage in cubic feet per second'),
    #data.table(fileName='UserDemand_gpd.txt', desc='User type water demand fraction in gallons per day'),
    #data.table(fileName='UserWithdrawal_gpd.txt', desc='User type water withdrawal in gallons per day'),
    #data.table(fileName='rain_in_obs.dat', desc='Observed rainfall in inches'),
    
    data.table(fileName='monthlySummary_Artificial_Drainage_cfs.txt', desc='Monthly Artificial drainage', jname='MonAve_ArtDrain_cfs'),
    data.table(fileName='monthlySummary_Evaporation_in.txt', desc='Monthly Evapotranspiration', jname='MonSum__Evap_in'),
    data.table(fileName='monthlySummary_Precipitation_in.txt', desc='Monthly Precipitation',jname='MonAve_Precip_in'),
    data.table(fileName='monthlySummary_FlowAtStreamNodes_cfs.txt', desc='Monthly Streamflow at node',jname='MonAve_FlowAtStreamNodes_cfs'),
    #data.table(fileName='monthlySummary_rain_in.dat', desc='Monthly summary of rainfall in inches',jname='ObsMonAve_Precip_in'),
    #data.table(fileName='monthlySummary_UserDemand_gpd.txt', desc='Monthly User Demand',jname='AveDaily_UserDemand_gpd'),
    #data.table(fileName='monthlySummary_UserWithdrawal_gpd.txt', desc='Monthly User Withdrawal'),
    #data.table(fileName='monthlySums_UserDemand_gpd.txt', desc='Monthly sum of User water demand',jname='MonSum_UserDemand_Gallons'),
    #data.table(fileName='monthlySums_UserWithdrawal_gpd.txt', desc='Monthly sum of User water withdrawal'),
    
    data.table(fileName='yearlySummary_Evaporation_in.txt', desc='Yearly Evapotranspiration',jname='AnnSumSubbasin_Evap_in'),
    data.table(fileName='yearlySummary_Precipitation_in.txt', desc='Yearly Precipitation',jname='AnnSumSubbasin_Precip_in'),
    #data.table(fileName='yearlySummary_rain_in_obs.dat', desc='Yearly summary of rainfall in inches'),
    #data.table(fileName='yearlySums_UserDemand_gpd.txt', desc='Yearly summary of user water demand in gallons per day'),
    #data.table(fileName='yearlySums_UserWithdrawal_gpd.txt', desc='Yearly summary of user water withdrawal in gallons per day'),
    
    data.table(fileName='AD_FASN_Summary', desc='Artificial draining and flow-at-Stream node summary'),
    data.table(fileName='streamflow_calibration_calib_cfs.dat', desc='Streamflow calibration data in cubic feet per second')),
                                 use.names=T, fill=T)

# append Sheet id
TableofContents.txt[,SheetName:=paste0('Sheet', row.names(TableofContents.txt))]


# print to txt files
for (j in as.vector(TableofContents.txt[['fileName']])){
    # "streamflow_calibration_valid_cfs.dat" excluded until additional patch work

    # start
    print(paste('start', j))

    # separate protocol for .dat files
    if(grepl('.dat',j)){
        write.table(j %>% gsub('_in_obs','',.) %>% paste0(., '_meta') %>% get(), file = j, append = F, sep = '\t', row.names = F, col.names = F, quote = F)
        write.table(j, file = j, append = T, sep = '\t', row.names = F, col.names = F)
    } else if (grepl('.txt', j)){
        write.table(j %>% get(), file = j, col.names = T, row.names = F, append = F)
    } else {
        write.table(j %>% get(), file = paste0(j,'.txt'), col.names = T, row.names = F, append = F)
    }

    # finished
    print(paste('finished', j))
}

tabs <- list(TableofContents.txt,
             DateTime_water_year.txt,
             UserType_table,
             
             Precipitation_in.txt,
             Evaporation_in.txt,
             FlowAtStreamNodes_cfs.txt,
             Artificial_Drainage_cfs.txt,
             #UserDemand_gpd.txt,
             #UserWithdrawal_gpd.txt,
             #rain_in_obs.dat,
             
             monthlySummary_Artificial_Drainage_cfs.txt,
             monthlySummary_Evaporation_in.txt,
             monthlySummary_Precipitation_in.txt,
             monthlySummary_FlowAtStreamNodes_cfs.txt,
             #monthlySummary_rain_in.dat,
             #monthlySummary_UserDemand_gpd.txt,
             #monthlySummary_UserWithdrawal_gpd.txt,
             #monthlySums_UserDemand_gpd.txt,
             #monthlySums_UserWithdrawal_gpd.txt,
             
             yearlySummary_Evaporation_in.txt,
             yearlySummary_Precipitation_in.txt,
             #yearlySummary_rain_in_obs.dat,
             #yearlySums_UserDemand_gpd.txt,
             #yearlySums_UserWithdrawal_gpd.txt,
             
             AD_FASN_Summary,
             streamflow_calibration_calib_cfs.dat)

# print these outputs to xlsx using these names
write_xlsx(tabs, path = paste(ModelSet_folder,'modelruns_summary.xlsx',sep='/'), col_names = TRUE) 

# END of Usable code (as far as I can tell)

In [ ]:
### The code chunk above is too long.  Also needs a loop if there is no user data. 

In [ ]:
# find meta lines for rain.dat
rain.dat_meta <- findmeta('rain.dat')

# read in rain.dat
rain.dat <- read_dat('rain.dat', rain.dat_meta)

In [ ]:
# apply conversion from mm to inches. Code-base taken from Precipitation.
# convert mm to inches
rain_in_obs.dat <- rain.dat %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(./25.4)) %>%
    merge(., DateTime_water_year.txt, by.x = 'Date', by.y = 'yyyymmdd') %>% 
    arrange(TimeStep) %>%
    select(-c(Date, year, day, Hour, hhmmss))

In [ ]:
# monthly average for the 9 water years
monthlySummary_rain_in.dat <- rain_in_obs.dat %>%
    # group by TimeStep and Month
    melt(., id.vars = c('TimeStep', 'water_year', 'month'), variable.name = 'zone_code', value.name = 'rain_in_obs') %>%
    mutate(zone_code = paste0('zone_',zone_code)) %>%

    # calculate total by month, water_year, and zone_code
    group_by(month, water_year, zone_code) %>%
    mutate(sumByMonthByWateryearByZonecode = sum(rain_in_obs)) %>%

    # calculate mean by month, and drainage
    group_by(month, zone_code) %>%
    mutate(mean_rain_in_obs = mean(sumByMonthByWateryearByZonecode)) %>%

    # calculate mean and sd by month
    group_by(month) %>%
    mutate(meanSumByMonth = mean(sumByMonthByWateryearByZonecode),
           meanByMonth = mean(rain_in_obs),
           sdByMonth = sd(rain_in_obs)) %>%

    # reformat the table to wide for each DrainageID
    select(-c(TimeStep, water_year, rain_in_obs, sumByMonthByWateryearByZonecode)) %>%
    dcast(., month+meanSumByMonth+meanByMonth+sdByMonth~zone_code, fun=mean, value.var = c('mean_rain_in_obs')) %>%
    data.table()

In [ ]:
# annual average for the 9 water years
yearlySummary_rain_in_obs.dat <- rain_in_obs.dat %>%
    # group by TimeStep and Month
    select(-c(TimeStep)) %>%
    melt(., id.vars = c('month', 'water_year'), variable.name = 'zone_code', value.name = 'rain_in_obs') %>%
    mutate(zone_code = paste0('zone_',zone_code)) %>%

    # calculate total by month, wateryear, and drainage
    group_by(month, water_year, zone_code) %>%
    mutate(sumByMonthByWateryearByZonecode = sum(rain_in_obs)) %>%

    # calculate mean by wateryear, and drainage
    group_by(water_year, zone_code) %>%
    mutate(meanByWateryearByZonecode = mean(sumByMonthByWateryearByZonecode)) %>%

    # calculate mean and sd by month
    group_by(water_year) %>%
    mutate(sumByWateryear = sum(rain_in_obs),
           meanByWateryear = mean(sumByMonthByWateryearByZonecode),
           sdByWateryear = sd(sumByMonthByWateryearByZonecode)) %>%

    # reformat the table to wide for each DrainageID
    select(-c(month, rain_in_obs, sumByMonthByWateryearByZonecode)) %>%
    dcast(., water_year + sumByWateryear + meanByWateryear + sdByWateryear ~ zone_code, fun = mean, value.var = c('meanByWateryearByZonecode')) %>%
    data.table()

In [ ]:
# find meta lines for streamflow_calibration.dat
streamflow_calibration.dat_meta <- findmeta('streamflow_calibration.dat')
col.Names <- streamflow_calibration.dat_meta %>% gsub("\\t+$","",.) %>% strsplit(' |\t') %>% tail(1) %>% unlist() %>% .[-c(1,2)] %>% .[.!=""]

In [ ]:
# read in streamflow_calibration.dat using readlines approach. read_dat was not working due to extra space characters.
# streamflow_calibration.dat <- read_dat('streamflow_calibration.dat', streamflow_calibration.dat_meta)
dat <- readLines('streamflow_calibration.dat', skip = length(streamflow_calibration.dat_meta)) %>% 
    .[-c(1:4)] %>% 
    gsub('^[[:space:]]+|[[:space:]]+$','', .) %>% 
    gsub('[[:space:]]+','\t', .) %>% 
    strsplit(., '\t') %>% 
    do.call(rbind, .) %>% 
    data.table()

nonNA <- colnames(dat)[which(colSums(is.na(dat)) != nrow(dat))]
dat <- dat %>% select(nonNA)
setnames(dat, colnames(dat), paste0('SF',col.Names))
streamflow_calibration.dat <- dat %>% 
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(as.numeric(.))) %>% data.table()

In [ ]:
# convert cms to cfs: 1cms = 35.3147 cfs
FlowAtStreamNodes_calib_cfs.txt <- FlowAtStreamNodes_cms.txt %>%
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    merge(., DateTime_calib.txt, by = 'TimeStep') %>% 
    mutate(FlowAtOutlet = Node1) %>% # 
    arrange(TimeStep) %>% 
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Node')) %>%
    data.table()


# convert cms to cfs: 1cms = 35.3147 cfs
FlowAtStreamNodes_valid_cfs.txt <- FlowAtStreamNodes_cms.txt %>%
    mutate_at(colnames(FlowAtStreamNodes_cms.txt)[!colnames(FlowAtStreamNodes_cms.txt) %in% c('Date','Hour', 'TimeStep')], funs(.*35.3147)) %>%
    merge(., DateTime_valid.txt, by = 'TimeStep', all.y = T) %>% 
    mutate(FlowAtOutlet = Node1) %>% # 
    arrange(TimeStep) %>% 
    select(TimeStep, yyyymmdd, year, month, day, water_year, contains('Node')) %>%
    data.table()

In [ ]:
# multiple flowatstreamnodes
# FlowAtStreamNodes_calib_cfs.txt
# FlowAtStreamNodes_valid_cfs.txt
# FlowAtStreamNodes_cfs.txt

# compare node4 (streamflow_calibration1) and node28 (streamflow_calibration2)
# the comparison between FlowAtStreamNodes (model output) and streamflow_calibration (obs)

# rename the metadata file
streamflow_calibration_calib_cfs.dat_meta <- streamflow_calibration.dat_meta

# adjust streamflow_calibration to cfs and filter for the calibration date range
streamflow_calibration_calib_cfs.dat <- streamflow_calibration.dat %>% 
    # convert cms to cfs
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    #mutate(SF152 = ifelse(SF152 == -999*35.3147, NA, SF152), 
    #       SF236 = ifelse(SF236 == -999*35.3147, NA, SF236)) %>%

    # add date sequence and convert to the %Y%m%d format
    mutate(date = seq(from = as.Date('20030613', format = '%Y%m%d'), by = 'day', length.out = nrow(streamflow_calibration.dat)), 
           date = format(date, '%Y%m%d')) %>%

    # filter for the respective model run dates
    filter(date %in% FlowAtStreamNodes_calib_cfs.txt[,yyyymmdd])


# # PATCH NEEDED: location observation files to use for imputation

# # obs data update for column 1
# BertrandStreamflow <- read.table(paste(ObsStreamflowData,'BertrandStreamflow.txt', sep='/'), sep = '\t', header = T) %>% 
#     mutate(Date = format(as.Date(Date, format = '%M/%d/%Y'), '%Y%m%d')) %>%
#     setnames(., colnames(.), c('Date', 'Flow_cfs')) %>%
#     data.table()

# CanadaStreamflow <- read.table(paste(ObsStreamflowData,'Daily__Sep-15-2016_10_14_39PM__ddf.txt', sep='/'), sep = '\t', header = T) %>% 
#     mutate(Date = format(as.Date(Date, format = '%M/%d/%Y'), '%Y%m%d')) %>%
#     data.table()


# # PATCH NEEDED: impute missing values using reference nodes

# # add the missing values
# streamflow_calibration_valid_cfs.dat <- streamflow_calibration_valid_cfs.dat %>%
#     mutate(SF152 = ifelse(is.na(SF152), BertrandStreamflow[match(date, Date), Flow_cfs], SF152),
#            SF236 = ifelse(is.na(SF236), CanadaStreamflow[match(date, Date), Value]*35.3147, SF236)) %>%
#     mutate(date = as.integer(date)) %>%
#     data.table()

# FlowAtStreamNodes_valid_cfs.txt <- FlowAtStreamNodes_valid_cfs.txt[yyyymmdd %in% streamflow_calibration_valid_cfs.dat[,date],]


# # PATCH NEEDED: compute nash-suttcliffe coefficient using reference nodes

# # operations linked to a reference node have been removed (e.g., SF152 and SF 236)
# NSEc1 <- NSE(sim = FlowAtStreamNodes_calib_cfs.txt[,Node4], obs = streamflow_calibration_calib_cfs.dat[,SF152], na.rm=T)
# NSEc2 <- NSE(sim = FlowAtStreamNodes_calib_cfs.txt[,Node28], obs = streamflow_calibration_calib_cfs.dat[,SF236], na.rm=T)


tab_names <- rbindlist(list(
    data.table(iname = 'monthlySummary_Artificial_Drainage_cfs.txt', jname = 'MonAve_ArtDrain_cfs'),
    data.table(iname = 'monthlySummary_Evaporation_in.txt', jname = 'MonSum__Evap_in'),
    data.table(iname = 'monthlySummary_FlowAtStreamNodes_cfs.txt', jname = 'MonAve_FlowAtStreamNodes_cfs'),
    data.table(iname = 'monthlySummary_Precipitation_in.txt', jname = 'MonAve_Precip_in'),
    data.table(iname = 'monthlySummary_rain_in.dat', jname = 'ObsMonAve_Precip_in'),
    data.table(iname = 'monthlySummary_UserDemand_gpd.txt', jname = 'AveDaily_UserDemand_gpd'),
    data.table(iname = 'monthlySums_UserDemand_gpd.txt', jname = 'MonSum_UserDemand_Gallons'),
    data.table(iname = 'yearlySummary_Evaporation_in.txt', jname = 'AnnSumSubbasin_Evap_in'),
    data.table(iname = 'yearlySummary_Precipitation_in.txt', jname = 'AnnSumSubbasin_Precip_in')), use.names = T, fill = T)

tab_names

In [ ]:
# multiple flowatstreamnodes
# FlowAtStreamNodes_calib_cfs.txt
# FlowAtStreamNodes_valid_cfs.txt
# FlowAtStreamNodes_cfs.txt

# compare node4 (streamflow_calibration1) and node28 (streamflow_calibration2)
# the comparison between FlowAtStreamNodes (model output) and streamflow_calibration (obs)

# rename the metadata file
streamflow_calibration_calib_cfs.dat_meta <- streamflow_calibration.dat_meta

# adjust streamflow_calibration to cfs and filter for the calibration date range
streamflow_calibration_calib_cfs.dat <- streamflow_calibration.dat %>% 
    # convert cms to cfs
    mutate_at(colnames(.)[!colnames(.) %in% c('Date','Hour','TimeStep')], funs(.*35.3147)) %>%
    #mutate(SF152 = ifelse(SF152 == -999*35.3147, NA, SF152), 
    #       SF236 = ifelse(SF236 == -999*35.3147, NA, SF236)) %>%

    # add date sequence and convert to the %Y%m%d format
    mutate(date = seq(from = as.Date('20030613', format = '%Y%m%d'), by = 'day', length.out = nrow(streamflow_calibration.dat)), 
           date = format(date, '%Y%m%d')) %>%
    # filter for the respective model run dates
    filter(date %in% FlowAtStreamNodes_calib_cfs.txt[,yyyymmdd])


# # PATCH NEEDED: generate calibration and validation summary tables

# # generate the NSE summary table
# NSE_Summary.txt <- rbind(
#     c('calibration', paste0(calib_start, '-', calib_end), NSEc1, NSEc2),
#     c('validation', paste0(valid_start, '-', valid_end), NSEv1, NSEv2)) %>%
#     data.frame() %>%
#     setnames(., colnames(.), c('NSE_mode','DateRange','Node4_SF152', 'Node28_SF236')) %>%
#     data.table()

In [ ]:
# # PATCH NEEDED: location observation files to use for imputation

# # obs data update for column 1
# BertrandStreamflow <- read.table(paste(ObsStreamflowData,'BertrandStreamflow.txt', sep='/'), sep = '\t', header = T) %>% 
#     mutate(Date = format(as.Date(Date, format = '%M/%d/%Y'), '%Y%m%d')) %>%
#     setnames(., colnames(.), c('Date', 'Flow_cfs')) %>%
#     data.table()

# CanadaStreamflow <- read.table(paste(ObsStreamflowData,'Daily__Sep-15-2016_10_14_39PM__ddf.txt', sep='/'), sep = '\t', header = T) %>% 
#     mutate(Date = format(as.Date(Date, format = '%M/%d/%Y'), '%Y%m%d')) %>%
#     data.table()

In [ ]:
# # PATCH NEEDED: impute missing values using reference nodes

# # add the missing values
# streamflow_calibration_valid_cfs.dat <- streamflow_calibration_valid_cfs.dat %>%
#     mutate(SF152 = ifelse(is.na(SF152), BertrandStreamflow[match(date, Date), Flow_cfs], SF152),
#            SF236 = ifelse(is.na(SF236), CanadaStreamflow[match(date, Date), Value]*35.3147, SF236)) %>%
#     mutate(date = as.integer(date)) %>%
#     data.table()

# FlowAtStreamNodes_valid_cfs.txt <- FlowAtStreamNodes_valid_cfs.txt[yyyymmdd %in% streamflow_calibration_valid_cfs.dat[,date],]

In [ ]:
# # PATCH NEEDED: compute nash-suttcliffe coefficient using reference nodes

# # operations linked to a reference node have been removed (e.g., SF152 and SF 236)
# NSEc1 <- NSE(sim = FlowAtStreamNodes_calib_cfs.txt[,Node4], obs = streamflow_calibration_calib_cfs.dat[,SF152], na.rm=T)
# NSEc2 <- NSE(sim = FlowAtStreamNodes_calib_cfs.txt[,Node28], obs = streamflow_calibration_calib_cfs.dat[,SF236], na.rm=T)

In [ ]:
# # PATCH NEEDED: generate calibration and validation summary tables

# # generate the NSE summary table
# NSE_Summary.txt <- rbind(
#     c('calibration', paste0(calib_start, '-', calib_end), NSEc1, NSEc2),
#     c('validation', paste0(valid_start, '-', valid_end), NSEv1, NSEv2)) %>%
#     data.frame() %>%
#     setnames(., colnames(.), c('NSE_mode','DateRange','Node4_SF152', 'Node28_SF236')) %>%
#     data.table()

In [ ]:
# print to txt files
for (j in c('Precipitation_in.txt', 
            'Evaporation_in.txt',
            'FlowAtStreamNodes_cfs.txt',
            'Artificial_Drainage_cfs.txt',
            'UserDemand_gpd.txt',
            'UserWithdrawal_gpd.txt',
            'DateTime_water_year.txt',
            'rain_in_obs.dat',
            "monthlySummary_Artificial_Drainage_cfs.txt",
            "monthlySummary_Evaporation_in.txt",
            "monthlySummary_Precipitation_in.txt",
            "monthlySummary_FlowAtStreamNodes_cfs.txt",
            "monthlySummary_UserDemand_gpd.txt",
            "monthlySummary_UserWithdrawal_gpd.txt",
            "yearlySummary_Evaporation_in.txt",
            "yearlySummary_Precipitation_in.txt",
            "AD_FASN_Summary",
            "monthlySums_UserDemand_gpd.txt",
            "monthlySums_UserWithdrawal_gpd.txt",
            "streamflow_calibration_calib_cfs.dat")){
    # "streamflow_calibration_valid_cfs.dat" excluded until additional patch work
    
    # start
    print(paste('start', j))
    
    # separate protocol for .dat files
    if(grepl('.dat',j)){
      write.table(j %>% gsub('_in_obs','',.) %>% paste0(., '_meta') %>% get(), file = j, append = F, sep = '\t', row.names = F, col.names = F, quote = F)
      write.table(j, file = j, append = T, sep = '\t', row.names = F, col.names = F)
    } else {
      write.table(j %>% get(), file = j, col.names = T, row.names = F, append = F)
    }
    
    # finished
    print(paste('finished', j))
}

In [ ]:
# Create the monthly and yearly summaries as a new xlsx workbook
# print to xlsx workbook

tabs <- list(AD_FASN_Summary,
             monthlySummary_Artificial_Drainage_cfs.txt,
             monthlySummary_Evaporation_in.txt,
             monthlySummary_FlowAtStreamNodes_cfs.txt,
             monthlySummary_Precipitation_in.txt,
             monthlySummary_rain_in.dat,
             monthlySummary_UserDemand_gpd.txt,
             monthlySummary_UserWithdrawal_gpd.txt,
             monthlySums_UserDemand_gpd.txt,
             monthlySums_UserWithdrawal_gpd.txt,
             UserType_table,
             yearlySummary_Evaporation_in.txt,
             yearlySummary_Precipitation_in.txt,
             yearlySummary_rain_in_obs.dat,
             yearlySums_UserDemand_gpd.txt,
             yearlySums_UserWithdrawal_gpd.txt)

# print these outputs to xlsx using these names
write_xlsx(tabs, path = paste0(ModelSet_folder,'modelruns_summary.xlsx'), col_names = TRUE)

In [ ]:
# compute the totalRecharge object
#source(file.path(WMon2018_Models, "RechargeCalculator_kbcb_jp.R"))

In [ ]:

# concerns:
# CatchID does not exist within Basins
# where is BasinArea_in2 created?
# where is Test1 initialized?

# functionalize the timesteploop
timesteploop <- function(TimeStep, totalRecharge_depot){
    for(TimeStep in TimeSteps$TimeStep){
        # CatchID + 1 will be used to adjust for TimeStep as the new column 1
    
        ### Look up P, E, and SR values for this daily timestep and convert to inches
        P <- Precipitation_mm[TimeStep, CatchID + 1]*0.0393701	#Precipitation (in)
        E <- Evaporation_mm[TimeStep, CatchID + 1]*0.0393701	#Evaporation (in)
        SR <- Surface_runoff_cms[TimeStep, CatchID + 1]*61024*60*60*24/Basins[CatchID, "BasinArea_in2"] #Surface runoff/Basin area (in)

        if(TimeStep==1){
          CDW <- as.numeric(0) ### Calculate change in depth to water (in)
          CSS <- as.numeric(0) ### Calculate change in soil storage term (in)
        } else {
          # take the difference between the current water depth and the previous water depth at this Drainage
          CDW <- (Depth_to_Water_mm[TimeStep, CatchID + 1] - Depth_to_Water_mm[TimeStep-1, CatchID + 1])*0.0393701
          CSS <- (Soil_storage_mm[TimeStep, CatchID + 1] - Soil_storage_mm[TimeStep-1, CatchID + 1])*0.0393701
        }

        ### Calculate daily recharge
        dailyRecharge = P-E-SR+CSS

        ### Calculate running total of daily recharge
        totalRecharge = totalRecharge + dailyRecharge

        # store the data
        totalRecharge_depot <- rbindlist(list(totalRecharge_depot, data.table(CatchID, TimeStep, dailyRecharge, totalRecharge)), use.names = T, fill = T)
    }
    return(totalRecharge_depot)
}

# daily iteration deposition
totalRecharge_depot <- data.table()
ptm <- Sys.time()

# for each subbasin
for (CatchID in Basins$CatchID) {
  
  ### Calculate average annual recharge (inches) for the basin ###
  totalRecharge=0

  # for each timestep
  totalRecharge_depot <- timesteploop(TimeStep, totalRecharge_depot)
  print(CatchID)
  ptm - Sys.time()
}


totalRecharge_depot <- read.table(file.path(Test1,'/totalRecharge_depot.txt'), sep = '\t', header = T)

# additional calculations
totalRecharge_depot <- totalRecharge_depot %>%
  # merge(., DateTime_water_year.txt, by = 'TimeStep') %>%
  
  # MonthlyRecharge total
  group_by(month, water_year, CatchID) %>%
  mutate(monthlyRecharge = sum(dailyRecharge)) %>%
  
  # WaterYear average AnnualRecharge
  group_by(water_year, CatchID) %>%
  mutate(averageAnnualRecharge = mean(monthlyRecharge)) %>%
  data.table() %>%
  
  group_by(CatchID) %>%
  mutate(averageAnnualRechargeByCatchID = mean(averageAnnualRecharge)) %>%
  data.table()



# print
write.table(totalRecharge_depot, 'totalRecharge_depot.txt', sep = '\t', row.names = F, col.names = T, append = F)
print(paste("Finished printing the RechargeCalculator within", getwd()))
